# Faire correspondre un modèle mathématique au code d'un solveur

## Introduction.
Dans ce notebook, on va analyser les méthodes pour faire correspondre un modèle mathématique donné à un code de solveur Python.
Autrement dit, comment passer de l'un à l'autre.

*** CREATION EN COURS - A VENIR !!***

<b>Notebook réalisé par GITHUB ESTELLE DERRIEN</b>

## Sommaire

1. Les symboles mathématique à connaitre.
2. Notre exemple de base
3. Extention de notre exemple et symbole de sommation.
4. L'itération et Les matrices.
5. On nous donne un modèle mathématique, par oû commencer ?
    - Exemple avec GLPK
    - Exemple avec PUlP
6. Et un modèle non llinéaire ?

# 1. Les signes à connaitre :

- Le symbole Σ (sigma), il s'apprends en classe de seconde et permet de simplifier l'ecriture d'une addition.

- Le symbole ∈ veut dire "appartient à", on s'en sert quand on a au préalable définit un ensemble S{x1,x2...xn}

- Le symbole ∀ veut dire "Pour tout"

- Une matrice Aij : i c'est la ligne, j c'est la colonne. Donc si on nous parle de la valeur A14, il s'agit de la valeur de la ligne 1 colonne 4.

- Une fonction objectif : elle est destinée à être minimisée ou maximisée, lorsque le solveur calcule les meilleures valeurs des variables de décision

- Les variables de décision : Une variable de décision contient une valeur qui sera ensuite calculée par le solveur pour être la meilleure valeur possible pour minimiser ou maximiser une fonction objectif ( C'est ça, l'optimisation)

- "Sujet à" ou "subject to" veut dire que la fonction objectif est sujette à des contraintes que l'on écrit sous la phrase "subject to".

- ℕ est l'ensemble des entiers positifs y compris le 0, N∗ omets le zéro.

# 2. Notre exemple de base :

On va prendre un modèle assez simple de mixe de production.

Dans les problèmes de débutant, au début, on a pas à utiliser le symbole de sommation, puisqu'on a généralement peu de variables, par déduction, on a pas à itérer avec le solveur.
C'est donc beaucoup plus simple à comprendre, au début.

# L'histoire 

- Une entreprise produit la voiture A et la voiture B.
- La voiture A nécessite 20 unités de fourniture K et 10 unités de fournitures Z.
- La voiture B nécessite 18 unités de fourniture K et 8 unités de fournitures Z.
- Il y a 5000 unités de fournitures K en stock et 6000 unités de fournitures Z en stock.
- La voiture A procure un bénéfice de 20000 euros, et la voiture B procure un bénéfice de 18000 euros.

Quelles voitures doit fournir l'entreprise afin de maximiser son bénéfice, sous contrainte de stocks ?


Donc là, dans ce problème, on voit bien qu'il y a peu de variables de décision en jeu et peu de contraintes.
Aucune raison d'exprimer le modèle mathématique avec des symboles de sommation, et de faire des itérations dans le code du solveur Python.

## On en déduit le modèle mathématique.

Soit

    A soit le nombre d'unités de voitures A produites  (C'est une variable de décision)
    B soit le nombre d'unités de de voitures B produites (C'est une variable de décision)

La fonction objectif R est de maximiser le bénéfice :<br>
Max(R) = 20000A + 18000B

Les contraintes sont les contraintes de stock :<br>

Stock de K<br>
20A + 18B <= 5000

Stock de Z<br>
10A + 8B <= 6000

## On écrit le modèle 

Max(R) = 20000A + 18000B<br>
S.T<br>
20A + 18B <= 5000<br>
10A + 8B <= 6000<br>
AVEC<br>
{A,B} ∈ ℕ<br>

## On écrit le code du solveur

In [9]:
# La méthode de base :

# -----------------------------------
# Import de Python Pulp
# -----------------------------------
from pulp import *

# -----------------------------------
# Type du problème
# -----------------------------------
# On choisit de résoudre un problème de Maximisation
model = LpProblem('Problem', LpMaximize)

# -----------------------------------
# Variables de décision
# -----------------------------------

A = LpVariable("A", lowBound=0, cat='Integer')   # Créer une variable x >= 0
B = LpVariable("B", lowBound=0, cat='Integer')   # Créer une variable y >= 0

# -----------------------------------
# Fonction objectif R
# -----------------------------------
model += 20000 * A + 18000  * B

# -----------------------------------
# Contraintes
# -----------------------------------

model += 20 * A + 18  * B <= 5000, "stock_produit_K"
model += 10 * A + 8  * B <= 6000, "stock_produit_Z"


# -----------------------------------
# Solution
# -----------------------------------
model.solve()
 
# On imprime les variables qui ont leur valeur optimisées
for v in model.variables():
    print(v.name, "=", v.varValue)

# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", value(model.objective))


A = 7.0
B = 270.0
Profit total maximisé =  5000000.0


## 3. Extention de notre exemple et symbole de sommation.

Admettons désormais que l'entreprise produise 12 voitures différentes et qu'elles nécessitent chacune une quantité de fourniture K et Z différentes, et qu'elles procurent chacune un bénéfice différent. 
L'objectif est toujours de déterminer quelles voitures il vaut mieux produire, afin de maximiser notre bénéfice sous contrainte de stock.
N'est il pas rébarbatif de retaper chaque valeur à la main ? Oui, c'est rébarbatif.

Ne serait-il pas plus pratique d'utiliser le symbole mathématique de sommation lorsque nous allons écrire le modèle mathématique, et d'itérer sur un tableau dans le code du solveur Python? Ceci afin de synthétiser l'expression de notre optimisation ?

Essayons donc désormais de modéliser notre modèle mathématique d'optimisation, avec le symbole de sommation, puis, on va itérer dans le code Python, pour obtenir notre optimisation linéaire finale.


## Tableau des fournitures nécessaires pour les 12 voitures.

In [10]:
# Voitures      A       B       C       D       E       F       G       H       I       J       K       L
# Fourniture K  20      18      9       20      22      10      20      12      15      22      21      5
# Fourniture Z  10      8       8       19      24      10      19      10      12      18      20      4
# Bénéfice      20000   18000   17000   21000   24000   18500   20500   18500   18900   21000   20500   8000 

## On en déduit le modèle mathématique.

Ecrivons cela en dur, afin de comprendre combien c'est rébarbatif.

Soit

    A soit le nombre d'unités de voitures A produites  (C'est une variable de décision)
    B soit le nombre d'unités de de voitures B produites (C'est une variable de décision)
    C soit le nombre d'unités de de voitures C produites (C'est une variable de décision)
    D soit le nombre d'unités de de voitures D produites (C'est une variable de décision)
    E soit le nombre d'unités de de voitures E produites (C'est une variable de décision)
    F soit le nombre d'unités de de voitures F produites (C'est une variable de décision)
    G soit le nombre d'unités de de voitures G produites (C'est une variable de décision)
    H soit le nombre d'unités de de voitures H produites (C'est une variable de décision)
    I soit le nombre d'unités de de voitures I produites (C'est une variable de décision)
    J soit le nombre d'unités de de voitures J produites (C'est une variable de décision)
    K soit le nombre d'unités de de voitures K produites (C'est une variable de décision)
    L soit le nombre d'unités de de voitures L produites (C'est une variable de décision)

La fonction objectif R est de maximiser le bénéfice :<br>
Max(R) = 20000A + 18000B + 17000C + 21000D + 24000E + 18500F + 20500G + 18500H + 18900I + 21000J + 20500K + 8000L

Les contraintes sont les contraintes de stock :<br>

Stock de K<br>
20A + 18B + 9C + 20D + 22E + 10F + 20G + 12H + 15I + 22J + 21K + 5L <= 5000

Stock de Z<br>
10A + 8B +8C + 19D + 24E + 10F + 19G + 10H + 12I + 18J + 20K + 4L <= 6000

## On écrit le modèle mathématique avec le symbole de sommation et les indices.

** ATTENTION CREATION ET CORRECTIONS EN COURS ET A VENIR ***

On voit bien que ci dessus, on a des sommes, et que les écrire à la main est propice à l'erreur.

- On remarque qu'il existe des ensembles que l'on peut indicer: Les voitures, Les fournitures K, les fournitures Z, les bénéfices.

- On remarque que notre Tableau des fournitures nécessaires pour les 12 voitures est semblable à une matrice Aij.

Rappel :  Soit Une matrice Aij : i c'est le numéro de la ligne, j c'est le numéro de la colonne. Donc si on nous parle de la valeur A14, il s'agit de la valeur de la ligne 1 colonne 4 de la matrice.

- On va donc créer un modèle mathématique, à l'aide d'ensembles, et de symboles de sommations, qui résumera de façon plus simple et plus compréhensible notre problème, ce sera la version synthétisée de notre optimisation.


Considérons désormais les ensembles :

- Vj pour les voitures, {A...L} indicé par j
- Kj pour les fournitures K, {20...5} indicé par j
- Zj pour les fournitures Z, {10...4} indicé par j
- Bj pour les bénéfices {20000...8000} indicé par j

On va déjà se servir uniquement des ensembles pour écrire le modèle mathématique, et pas de la matrice; ça, on le fera dans un second temps, car c'est plus compliqué à lire.


Si on veut réécrire la fonction objectif synthétisée qui maximise le bénéfice de la somme des voitures: <br> 
Max(R) = 20000A + 18000B + 17000C + 21000D + 24000E + 18500F + 20500G + 18500H + 18900I + 21000J + 20500K + 8000L

On l'écrit : 

$$
Max (R) = \sum_{j=1}^{V} Bj.Vj
$$

Si on veut réécrire la première contrainte de stock K, de façon synthétisée :<br> 
20A + 18B + 9C + 20D + 22E + 10F + 20G + 12H + 15I + 22J + 21K + 5L <= 5000

On l'écrit : 
$$
 \sum_{j=1}^{K} Kj.Vj <= 5000
$$

Si on veut réécrire la première contrainte de stock Z, de façon synthétisée :<br> 
10A + 8B +8C + 19D + 24E + 10F + 19G + 10H + 12I + 18J + 20K + 4L <= 6000

On l'écrit : 
$$
 \sum_{j=1}^{Z} Zj.Vj <= 6000
$$
